In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch import autograd

from utils_2 import mnist

In [2]:
train_loader, test_loader = mnist()

In [3]:
def CrossEntropy(x, y):
    m = y.shape[0]
    p = F.softmax(x)
    log_likelihood = -1*torch.log(p[range(m),y])
    loss = torch.sum(log_likelihood) / m
    return loss

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 10)
        self.optim = optim.Adam(self.parameters(), lr=0.01)
        
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.sigmoid(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def loss(self, output, target, **kwargs):
        self._loss = CrossEntropy(output, target)
        return self._loss

In [5]:
def train(epoch, models):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data, target
        for model in models:
            model.optim.zero_grad()
            output = model(data)
            loss = model.loss(output, target)
            loss.backward()
            model.optim.step()
            
        if batch_idx % 200 == 0:
            line = 'Train Epoch: {} [{}/{} ({:.0f}%)]\tLosses '.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader))
            losses = ' '.join(['{}: {:.6f}'.format(i, m._loss.item()) for i, m in enumerate(models)])
            print(line + losses)
            
    else:
        batch_idx += 1
        line = 'Train Epoch: {} [{}/{} ({:.0f}%)]\tLosses '.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader))
        losses = ' '.join(['{}: {:.6f}'.format(i, m._loss.item()) for i, m in enumerate(models)])
        print(line + losses)

In [6]:
models = [Net()]

In [7]:
avg_lambda = lambda l: 'Loss: {:.4f}'.format(l)
acc_lambda = lambda c, p: 'Accuracy: {}/{} ({:.0f}%)'.format(c, len(test_loader.dataset), p)
line = lambda i, l, c, p: '{}: '.format(i) + avg_lambda(l) + '\t' + acc_lambda(c, p)

def test(models):
    test_loss = [0]*len(models)
    correct = [0]*len(models)
    with torch.no_grad():
        for data, target in test_loader:
            output = [m(data) for m in models]
            for i, m in enumerate(models):
                test_loss[i] += m.loss(output[i], target, size_average=False).item() # sum up batch loss
                pred = output[i].data.max(1, keepdim=True)[1] # get the index of the max log-probability
                correct[i] += pred.eq(target.data.view_as(pred)).cpu().sum()
    
    for i in range(len(models)):
        test_loss[i] /= len(test_loader.dataset)
    correct_pct = [100. * c / len(test_loader.dataset) for c in correct]
    lines = '\n'.join([line(i, test_loss[i], correct[i], correct_pct[i]) for i in range(len(models))]) + '\n'
    report = 'Test set:\n' + lines
    
    print(report)

In [ ]:
for epoch in range(1, 21):
    train(epoch, models)
    test(models)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  This is separate from the ipykernel package so we can avoid doing imports until


Train Epoch: 1 [0/60000 (0%)]	Losses 0: 0.077111
Train Epoch: 1 [10000/60000 (17%)]	Losses 0: 0.037792
Train Epoch: 1 [20000/60000 (33%)]	Losses 0: 0.287029
Train Epoch: 1 [30000/60000 (50%)]	Losses 0: 0.092039
Train Epoch: 1 [40000/60000 (67%)]	Losses 0: 0.083597
Train Epoch: 1 [50000/60000 (83%)]	Losses 0: 0.085333
Train Epoch: 1 [60000/60000 (100%)]	Losses 0: 0.143636
Test set:
0: Loss: 0.0034	Accuracy: 9475/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Losses 0: 0.189839
Train Epoch: 2 [10000/60000 (17%)]	Losses 0: 0.143986
Train Epoch: 2 [20000/60000 (33%)]	Losses 0: 0.215731
Train Epoch: 2 [30000/60000 (50%)]	Losses 0: 0.349358
Train Epoch: 2 [40000/60000 (67%)]	Losses 0: 0.132694
Train Epoch: 2 [50000/60000 (83%)]	Losses 0: 0.110149
Train Epoch: 2 [60000/60000 (100%)]	Losses 0: 0.106731
Test set:
0: Loss: 0.0033	Accuracy: 9498/10000 (94%)

Train Epoch: 3 [0/60000 (0%)]	Losses 0: 0.280129
Train Epoch: 3 [10000/60000 (17%)]	Losses 0: 0.097335
Train Epoch: 3 [20000/60000 (33%)]	Losses